In [1]:
from typing import Any, Dict, List

import torch.nn as nn
from dualcodec.model_codec.dac_model import DAC                    
from math import ceil, floor

/home/vansh/dualcodec-exp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Layer = Dict[str, Any]

In [ ]:
def _add_layer(lst: List[Layer], name: str, mod: nn.Module, ltype: str) -> None:
    lst.append(
        dict(
            name=name,
            type=ltype,                      # 'conv' | 'tconv'
            k=mod.kernel_size[0],
            s=mod.stride[0] if hasattr(mod, stride) else 1,
            d=mod.dilation[0],
            p=mod.padding[0],
            in_ch=mod.in_channels,
            out_ch=mod.out_channels,
        )
    )

In [4]:
def collect_encoder_layers(enc: nn.Module, strides: List[int]) -> List[Layer]:
    layers: List[Layer] = []

    # 0. input conv
    _add_layer(layers, "EncInputConv", enc.block[0], "conv")

    idx = 1
    for blk_no, stride in enumerate(strides, 1):
        enc_blk = enc.block[idx]         # EncoderBlock N
        idx += 1

        unit_no = 0
        for sub in enc_blk.block:
            # ResidualUnit
            if hasattr(sub, "block"):    # picks up ResidualUnit
                unit_no += 1
                _add_layer(
                    layers,
                    f"EncBlock{blk_no}-Unit{unit_no}-Conv1",
                    sub.block[1],
                    "conv",
                )
                _add_layer(
                    layers,
                    f"EncBlock{blk_no}-Unit{unit_no}-Conv2",
                    sub.block[3],
                    "conv",
                )
            # down-sampling conv is plain Conv1d
            elif isinstance(sub, nn.Conv1d):
                _add_layer(
                    layers,
                    f"EncBlock{blk_no}-DownsampleConv",
                    sub,
                    "conv",
                )

    _add_layer(layers, "EncFinalConv", enc.block[-1], "conv")
    return layers

In [5]:
def collect_decoder_layers(dec: nn.Module, rates: List[int]) -> List[Layer]:
    layers: List[Layer] = []

    _add_layer(layers, "DecInputConv", dec.model[0], "conv")

    cursor = 1
    for blk_no, _ in enumerate(rates, 1):
        dec_blk = dec.model[cursor]      # DecoderBlock N
        cursor += 1

        _add_layer(
            layers,
            f"DecBlock{blk_no}-UpsampleConv",
            dec_blk.block[1],
            "tconv",
        )

        unit_no = 0
        for sub in dec_blk.block:
            if hasattr(sub, "block"):    # ResidualUnit
                unit_no += 1
                _add_layer(
                    layers,
                    f"DecBlock{blk_no}-Unit{unit_no}-Conv1",
                    sub.block[1],
                    "conv",
                )
                _add_layer(
                    layers,
                    f"DecBlock{blk_no}-Unit{unit_no}-Conv2",
                    sub.block[3],
                    "conv",
                )

    _add_layer(layers, "DecFinalConv", dec.model[-2], "conv")
    return layers

In [12]:
def annotate_for_index(layers: List[Layer], t_out: int) -> None:
    """
    Annotate each layer with receptive field info, working backwards from t_out.
    Jumps are computed as the product of strides up to that layer.
    L and R are computed recursively from the output index.
    """
    print(f"t_out: {t_out}")
    n = len(layers)
    # Compute jumps for each layer (J_l = product of strides up to l)
    jumps = []
    J = 1.0
    for layer in layers:
        jumps.append(J)
        if layer["type"] == "conv":
            J *= layer["s"]
        else:
            J /= layer["s"]

    # Work backwards to compute L and R for each layer
    L = R = t_out
    prev_len = 1
    for i in reversed(range(n)):
        layer = layers[i]
        k, s, d, p = layer["k"], layer["s"], layer["d"], layer["p"]
        J = jumps[i]
        if layer["type"] == "conv":
            L_new = L - p * J
            R_new = R + ((k - 1) * d - p) * J
        else:  # transposed conv
            # Use the same formula as in the original code for tconv
            L_new = ceil(L/J + (p - (k - 1) * d) / s) * J
            R_new = floor(R/J + p / s) * J
        rf_len = R_new - L_new + 1
        layer.update(
            dict(jump=J, L=L_new, R=R_new, rf_len=rf_len, delta_len=rf_len - prev_len)
        )
        prev_len = rf_len
        L, R = L_new, R_new


def print_table(layers: List[Layer], title: str, t_val: int) -> None:
    hdr = ("Layer", "In", "Out", "k", "s", "d", "pad",
           "jump", "L", "R", "len", "Δlen")
    print(f"\n{title}  (t = {t_val})\n" + "=" * 156)
    print(f"{hdr[0]:<50} {hdr[1]:>4} {hdr[2]:>4} "
          f"{hdr[3]:>3} {hdr[4]:>3} {hdr[5]:>3} {hdr[6]:>4} "
          f"{hdr[7]:>10} {hdr[8]:>9} {hdr[9]:>9} {hdr[10]:>6} {hdr[11]:>6}")
    print("-" * 156)
    for l in layers:
        print(f"{l['name']:<50} {l['in_ch']:>4} {l['out_ch']:>4} "
              f"{l['k']:>3} {l['s']:>3} {l['d']:>3} {l['p']:>4} "
              f"{l['jump']:>10.6f} {l['L']:>9} {l['R']:>9} "
              f"{l['rf_len']:>6} {l['delta_len']:>6}")

In [13]:
dac = DAC(decoder_rates=[2,8,6,5,4]).eval()

dec_layers = collect_decoder_layers(dac.decoder, dac.decoder_rates)
annotate_for_index(dec_layers, 0)
print_table(dec_layers, "DECODER", 0)

/home/vansh/dualcodec-exp/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


AttributeError: 'CausalWNConv1d' object has no attribute 'stride'

In [9]:
dac = DAC(decoder_rates=[2,8,6,5,4], padding_ratio = 0).eval()

dec_layers = collect_decoder_layers(dac.decoder, dac.decoder_rates)
annotate_for_index(dec_layers, 0)
print_table(dec_layers, "DECODER", 0)

TypeError: DAC.__init__() got an unexpected keyword argument 'padding_ratio'